# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1uAUJGEUzfNj6OsWNAimnYCw7eKaHhMUfU1MTj9YwYw4/edit?usp=sharing), [grading rubric](https://docs.google.com/document/d/1hKuRWqFcIdhOkow3Nljcm7PXzIkoa9c_aHkMKZDxWa0/edit?usp=sharing)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an outline to help you with your own approach.**_

## Project Setup

In [1]:
# all import statements needed for the project, for example:
import numpy as np
import math
import os
import bs4
import sqlite3
import matplotlib.pyplot as plt
import sqlalchemy as db

import requests
from bs4 import BeautifulSoup
import pyarrow.parquet as pq
import pandas as pd
import wget
import geopandas as gpd
import xorbits
import plotly.express as px

In [36]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "data/taxi_zones"

TAXI_ZONES_SHAPEFILE = "taxi_zones.shp"


CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite://project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
uber_path = "/Users/xuzh/Desktop/TFA/uber_rides_sample.csv"
weather_path1 = "/Users/xuzh/Desktop/TFA/"
yellowtaxi_path = "/Users/xuzh/Desktop/TFA/"

In [37]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [38]:
#load the taxi shp file
def load_taxi_zones(shapefile):
    s=gpd.read_file(shapefile)
    return s



In [39]:
#check if the input location id is within the required zones
def lookup_coords_for_taxi_zone_id( loaded_taxi_zones,zone_loc_id=None):
    #print(loaded_taxi_zones)
    s=load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    df=pd.DataFrame()
    df['ID']=s['OBJECTID']
    #print(df["ID"])
    if loaded_taxi_zones in df['ID'].unique():
        return True
    return False

In [40]:
#test cell
print(lookup_coords_for_taxi_zone_id(299))

False


### Process Taxi Data

In [44]:
def get_all_urls_from_taxi_page(taxi_page):
    # send a request to the taxi page and get its HTML content
    response = requests.get(taxi_page)
    content = response.content
    
    # create a BeautifulSoup object and find all links on the page
    soup = BeautifulSoup(content, 'html.parser')
    
    links = soup.find_all('a')
    
    # extract URLs from the links and store them in a list
    urls = []
    for link in links:
        url = link.get('href')
        if url and url.endswith('.parquet') and 'yellow_tripdata' in url:
            urls.append(url)

    return urls

In [45]:
#Test cell: get_all_urls_from_taxi_page will not return a empty value
assert get_all_urls_from_taxi_page(TAXI_URL) != None


In [ ]:
def filter_taxi_parquet_urls(all_urls):
    #down data from years 2008 to 2015
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    l=list(range(2009,2016))
    l=[str(x) for x in l]
    urls_fillter=[]
    for url in all_urls:
        for flag in l:
            if (flag in url):
                urls_fillter.append(url)
    print(urls_fillter)
    return urls_fillter

In [48]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

In [50]:
def get_taxi_data():
    #download the required data
    all_urls = filter_taxi_parquet_urls(TAXI_URL)
    for url in all_urls:
        wget.download(url,"/Users/xuzh/Desktop/TFA/queries"+url.split('/')[-1])

In [52]:
#This step may take very long, depending on your download speed
get_taxi_data()

KeyboardInterrupt: 

In [17]:
#clean and filter yellow taxi data from 2009
def get_yellowtaxi_9(path):
    df = pd.read_parquet(path, engine='pyarrow')
    new_df=df.drop(['vendor_name', 'Rate_Code',
 'store_and_forward', 'Payment_Type' ,'Fare_Amt',
 'surcharge' ,'mta_tax','Tolls_Amt' ,'Total_Amt'], axis=1)
    modi_df = new_df[(40.560445 < new_df['Start_Lat']) & (new_df['Start_Lat'] < 40.908524) &
                     (40.560445 < new_df['End_Lat']) & (new_df['End_Lat'] < 40.908524) &
                     (-73.717047 > new_df['End_Lon']) & (new_df['End_Lon'] > -74.242330) &
                     (-73.717047 > new_df['Start_Lon']) & (new_df['Start_Lon'] > -74.242330)]


    df=modi_df.dropna(axis=0, how='any', inplace=False)
    df = df.drop(['Start_Lon','Start_Lat','End_Lon','End_Lat'], axis=1)
    df=df.sample(n=2380,random_state=1)
    df["Trip_Pickup_Date"]=pd.to_datetime(df['Trip_Pickup_DateTime']).dt.date
    df['Trip_Pickup_DateTime'] = pd.to_datetime(df['Trip_Pickup_DateTime'])
    df['Trip_Pickup_DateHour'] = df['Trip_Pickup_DateTime'].dt.strftime('%H')
    df=df.drop(['Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime',], axis=1)
    df.columns = ['passenger_count', 'trip_distance', 'tip_amount','Trip_Pickup_Date', 'Trip_Pickup_DateHour']
    # print(df.dtypes)
    # print(df.head())
    return df
#clean and filter yellow taxi data from 2012
def get_yellowtaxi_12(path):
    df = pd.read_parquet(path, engine='pyarrow')
    # df=df.sample(n=3000,random_state=1)
    new_df = df.drop(['tpep_dropoff_datetime', 'RatecodeID','VendorID',
                      'store_and_fwd_flag', 'payment_type', 'fare_amount',
                      'extra', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge','mta_tax',
                      'airport_fee'], axis=1)
    # print(new_df.head())
    # new_df["pul"] = new_df['PULocationID'].apply(lookup_coords_for_taxi_zone_id)
    # new_df["dol"] = new_df['DOLocationID'].apply(lookup_coords_for_taxi_zone_id)
    modi_df = new_df[(new_df['PULocationID'] <= 263) & (new_df['DOLocationID'] <= 263)]
    # print(new_df.head())
    df = modi_df.dropna(axis=0, how='any', inplace=False)
    df = df.drop(['PULocationID', 'DOLocationID', ], axis=1)
    df["Trip_Pickup_Date"] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
    df['Trip_Pickup_DateTime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['Trip_Pickup_DateHour'] = df['tpep_pickup_datetime'].dt.strftime('%d')
    df=df[(df['Trip_Pickup_DateHour'])]
    df = df.drop(['Trip_Pickup_DateTime'], axis=1)
    df = df.drop(['tpep_pickup_datetime'], axis=1)
    #print(df.head())
    df.columns = ['passenger_count', 'trip_distance', 'tip_amount','Trip_Pickup_Date', 'Trip_Pickup_DateHour']
    return df
#clean and filter yellow taxi data from 2015
def get_yellowtaxi_15(path):
    df = pd.read_parquet(path, engine='pyarrow')
    # df=df.sample(n=3000,random_state=1)
    new_df = df.drop(['tpep_dropoff_datetime', 'RatecodeID','VendorID',
                      'store_and_fwd_flag', 'payment_type', 'fare_amount',
                      'extra', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge','mta_tax','trip_distance', 'tip_amount',
                      'airport_fee'], axis=1)
   # print(new_df.head())
    # new_df["pul"] = new_df['PULocationID'].apply(lookup_coords_for_taxi_zone_id)
    # new_df["dol"] = new_df['DOLocationID'].apply(lookup_coords_for_taxi_zone_id)
    modi_df = new_df[(new_df['PULocationID'] <= 263) & (new_df['DOLocationID'] <= 263)]
    # print(new_df.head())
    df = df.sample(n=2380, random_state=1)
    df = modi_df.dropna(axis=0, how='any', inplace=False)
    #df = df.drop(['PULocationID', 'DOLocationID', ], axis=1)
    # df["Trip_Pickup_Date"] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
    # df = df.drop(['Trip_Pickup_DateTime'], axis=1)
    df = df.drop(['PULocationID'], axis=1)
    #df = df.drop(['tpep_pickup_datetime'], axis=1)
    #print(df.head())
    #df.columns = ['passenger_count',  'Trip_Pickup_DateHour']

    return df

In [18]:
#clean and filter yellow taxi data from 2010

def get_yellowtaxi_10(path):
    df = pd.read_parquet(path, engine='pyarrow')
    #print(df.head())
    new_df=df.drop(['vendor_id', 'rate_code',
 'store_and_fwd_flag', 'payment_type' ,'fare_amount',
 'surcharge' ,'mta_tax','tolls_amount' ,'total_amount'], axis=1)
    modi_df = new_df[(40.560445 < new_df['pickup_latitude']) & (new_df['pickup_latitude'] < 40.908524) &
                     (40.560445 < new_df['dropoff_latitude']) & (new_df['dropoff_latitude'] < 40.908524) &
                     (-73.717047 > new_df['dropoff_longitude']) & (new_df['dropoff_longitude'] > -74.242330) &
                     (-73.717047 > new_df['pickup_longitude']) & (new_df['pickup_longitude'] > -74.242330)]


    df=modi_df.dropna(axis=0, how='any', inplace=False)
    df = df.drop(['pickup_latitude','dropoff_latitude','dropoff_longitude','pickup_longitude'], axis=1)
    df=df.sample(n=2380,random_state=1)
    df["Trip_Pickup_Date"]=pd.to_datetime(df['pickup_datetime']).dt.date
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df['Trip_Pickup_DateHour'] = df['pickup_datetime'].dt.strftime('%H')
    df=df.drop(['pickup_datetime', 'dropoff_datetime',], axis=1)
    #print(df.head())
    df.columns = ['passenger_count', 'trip_distance', 'tip_amount','Trip_Pickup_Date', 'Trip_Pickup_DateHour']
    # print(df.dtypes)
    # print(df.head())
    return df

In [19]:
#clean and filter yellow taxi data from 2011
def get_yellowtaxi_11(path):
    df = pd.read_parquet(path, engine='pyarrow')
    # df=df.sample(n=3000,random_state=1)
    new_df = df.drop(['tpep_dropoff_datetime', 'RatecodeID','VendorID',
                      'store_and_fwd_flag', 'payment_type', 'fare_amount',
                      'extra', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge','mta_tax',
                      'airport_fee'], axis=1)
    # print(new_df.head())
    # new_df["pul"] = new_df['PULocationID'].apply(lookup_coords_for_taxi_zone_id)
    # new_df["dol"] = new_df['DOLocationID'].apply(lookup_coords_for_taxi_zone_id)
    modi_df = new_df[(new_df['PULocationID'] <= 263) & (new_df['DOLocationID'] <= 263)]
    # print(new_df.head())
    df = modi_df.dropna(axis=0, how='any', inplace=False)
    df = df.drop(['PULocationID', 'DOLocationID', ], axis=1)
    df = df.sample(n=2380, random_state=1)
    df["Trip_Pickup_Date"] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
    df['Trip_Pickup_DateTime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['Trip_Pickup_DateHour'] = df['tpep_pickup_datetime'].dt.strftime('%H')
    df = df.drop(['Trip_Pickup_DateTime'], axis=1)
    df = df.drop(['tpep_pickup_datetime'], axis=1)
    #print(df.head())
    df.columns = ['passenger_count', 'trip_distance', 'tip_amount','Trip_Pickup_Date', 'Trip_Pickup_DateHour']
    # print(df.dtypes)
    # print(df.head())
    return df

In [20]:
def get_add_yellowtaxi():
    # urls=[r'C:\py-work\work23\projects\c0428_2\\queriesyellow_tripdata_2015-05.parquet',
    #       r'C:\py-work\work23\projects\c0428_2\\queriesyellow_tripdata_2009-05.parquet',
    #       r'C:\py-work\work23\projects\c0428_2\\queriesyellow_tripdata_2013-05.parquet',
    #       r'C:\py-work\work23\projects\c0428_2\\queriesyellow_tripdata_2015-01.parquet',]
    urls=filter_taxi_parquet_urls(get_all_urls_from_taxi_page(TAXI_URL))
    df=[]
    # df.columns = ['passenger_count', 'trip_distance', 'tip_amount','Trip_Pickup_Date', 'Trip_Pickup_DateHour']
    # pd.concat([df,])
    for url in urls:
        filename=yellowtaxi_path+'queries'+url.split('/')[-1]
        print(filename)
        if '2009' in filename:
            write_yellow(get_yellowtaxi_9(filename))
            #df.append(get_yellowtaxi_9(filename))
        elif '2010' in filename:
            write_yellow(get_yellowtaxi_10(filename))
            #df.append(get_yellowtaxi_10(filename))
        else:
            write_yellow(get_yellowtaxi_11(filename))
            #df.append(get_yellowtaxi_11(filename))
            # pd.concat([df,get_yellowtaxi_11(filename)],axis=0)
    #pd.concat(df,axis=0)


### Processing Uber Data

In [21]:
def get_uberdata(path):
    df = pd.read_csv(path, low_memory=False)
    new_df = df.drop(['Unnamed: 0', 'fare_amount', 'key'], axis=1)
    modi_df = new_df[(40.560445 < new_df['pickup_latitude']) & (new_df['pickup_latitude'] < 40.908524) &
                     (40.560445 < new_df['dropoff_latitude']) & (new_df['dropoff_latitude'] < 40.908524) &
                     (-73.717047 > new_df['dropoff_longitude']) & (new_df['dropoff_longitude'] > -74.242330) &
                     (-73.717047 > new_df['pickup_longitude']) & (new_df['pickup_longitude'] > -74.242330)]
    modi_df = modi_df.reset_index(drop=True)
    modi_df['pickup_datetime']=pd.to_datetime(modi_df['pickup_datetime'])
    # modi_df['pickup_datetime']=modi_df['pickup_datetime'].dt.strftime('%Y-%m-%d %H')

    modi_df=modi_df.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1)
    return modi_df

In [22]:
uber_df=get_uberdata(uber_path)
print(uber_df.head())

            pickup_datetime  passenger_count
0 2015-05-07 19:52:06+00:00                1
1 2009-07-17 20:04:56+00:00                1
2 2009-08-24 21:45:00+00:00                1
3 2009-06-26 08:22:21+00:00                3
4 2014-08-28 17:47:00+00:00                5


### Processing Weather Data

In [23]:
#Read a local weather data csv file, normalize column names, and put into a dataframe
def get_hour_weather(path):
    df = pd.read_csv(path, low_memory=False)
    new_df = df[["DATE", "REPORT_TYPE", "HourlyWindSpeed", "HourlyPrecipitation"]]

    new_df['DATE']=pd.to_datetime(new_df['DATE'])
    new_df['nDATE']=new_df['DATE'].dt.strftime('%M')
    new_df['ndate']=new_df['DATE']
    modi_df = new_df[(new_df['nDATE']=='51')]
    modi_df = modi_df.reset_index(drop=True)
    modi_df = modi_df.drop(['REPORT_TYPE','nDATE'], axis=1)
    #print(modi_df.head())
    return modi_df

In [24]:
#Test cell
#get_hour_weather('/Users/xuzh/Desktop/TFA/2009_weather.csv')

In [25]:
#Normalize data in Hourly Precipitation
def get_day_weather(weather):
    weather['DATE'] = pd.to_datetime(weather['DATE']).dt.date
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace(np.nan, 0)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("T", 0)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.01s", 0.01)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.02s", 0.02)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.03s", 0.03)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.04s", 0.04)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.05s", 0.05)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.06s", 0.06)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.07s", 0.07)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.08s", 0.08)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.09s", 0.09)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.10s", 0.1)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.11s", 0.11)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.12s", 0.12)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.13s", 0.13)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.14s", 0.14)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.15s", 0.15)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.16s", 0.16)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.17s", 0.17)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.18s", 0.18)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.19s", 0.19)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.20s", 0.2)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.21s", 0.21)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.22s", 0.22)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.23s", 0.23)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.24s", 0.24)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.25s", 0.25)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.26s", 0.26)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.27s", 0.27)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.28s", 0.28)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.29s", 0.29)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.30s", 0.3)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.31s", 0.31)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.32s", 0.32)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.33s", 0.33)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.34s", 0.34)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.35s", 0.35)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.36s", 0.36)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.37s", 0.37)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.38s", 0.38)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.39s", 0.39)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.40s", 0.4)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.41s", 0.41)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.42s", 0.42)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.43s", 0.43)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.44s", 0.44)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.45s", 0.45)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.46s", 0.46)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.47s", 0.47)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.48s", 0.48)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.49s", 0.49)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.50s", 0.5)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.51s", 0.51)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.52s", 0.52)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.53s", 0.53)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.54s", 0.54)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.55s", 0.55)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.56s", 0.56)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.57s", 0.57)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.58s", 0.58)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.59s", 0.59)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.60s", 0.6)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.61s", 0.61)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.62s", 0.62)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.63s", 0.63)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.64s", 0.64)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.65s", 0.65)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.66s", 0.66)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.67s", 0.67)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.68s", 0.68)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.69s", 0.69)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.70s", 0.7)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.71s", 0.71)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.72s", 0.72)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.73s", 0.73)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.74s", 0.74)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.75s", 0.75)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.76s", 0.76)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.77s", 0.77)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.78s", 0.78)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.79s", 0.79)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.80s", 0.8)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.81s", 0.81)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.82s", 0.82)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.83s", 0.83)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.84s", 0.84)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.85s", 0.85)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.86s", 0.86)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.87s", 0.87)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.88s", 0.88)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.89s", 0.89)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.90s", 0.9)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.91s", 0.91)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.92s", 0.92)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.93s", 0.93)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.94s", 0.94)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.95s", 0.95)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.96s", 0.96)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.97s", 0.97)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.98s", 0.98)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].replace("0.99s", 0.99)
    weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].astype(float)
    #print(weather)
    weather_day1 = weather.groupby('DATE')['HourlyWindSpeed'].mean()
    weather_day2 = weather.groupby('DATE')['HourlyPrecipitation'].sum()
    weather_day1 = weather_day1.to_frame()
    weather_day2 = weather_day2.to_frame()
    weather_day1['HourlyPrecipitation']=weather_day2['HourlyPrecipitation']
    #print(weather_day1)
    return weather_day1


In [26]:
#Load and clean weather data from 2009 to 2015
def load_and_clean_weather_data(file_path):
    weather0 = get_hour_weather(file_path+'2009_weather.csv')
    a0 = get_day_weather(weather0)
    l=list(range(2009,2016))
    l=[str(x) for x in l]
    for i in l:
        file_name=file_path+f'{i}_weather.csv'
        print(file_name)
        weather1 = get_hour_weather(file_name)
        #print(weather1)
        a = get_day_weather(weather1)
        #print(a)
        #write_hourlyweather(weather1)
        # write_daylyweather(a)


        pd.concat([weather0,weather1],axis=0)
        pd.concat([a0,a],axis=0)

    # create two dataframes with hourly & daily data from every month
    hourly_data = weather0
    daily_data = a0

    return hourly_data, daily_data


In [27]:
#test cell
#load_and_clean_weather_data(weather_path1)

In [28]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data(weather_path1)
print(hourly_weather_data.head())
print(daily_weather_data.head())

<ipython-input-23-1551ad0c6435>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['DATE']=pd.to_datetime(new_df['DATE'])
<ipython-input-23-1551ad0c6435>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['nDATE']=new_df['DATE'].dt.strftime('%M')
<ipython-input-23-1551ad0c6435>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

/Users/guowenlu/Desktop/TFA/2009_weather.csv
/Users/guowenlu/Desktop/TFA/2010_weather.csv
/Users/guowenlu/Desktop/TFA/2011_weather.csv
/Users/guowenlu/Desktop/TFA/2012_weather.csv
/Users/guowenlu/Desktop/TFA/2013_weather.csv
/Users/guowenlu/Desktop/TFA/2014_weather.csv
/Users/guowenlu/Desktop/TFA/2015_weather.csv
         DATE  HourlyWindSpeed  HourlyPrecipitation               ndate
0  2009-01-01             18.0                  0.0 2009-01-01 00:51:00
1  2009-01-01             18.0                  0.0 2009-01-01 01:51:00
2  2009-01-01             18.0                  0.0 2009-01-01 02:51:00
3  2009-01-01              8.0                  0.0 2009-01-01 03:51:00
4  2009-01-01             11.0                  0.0 2009-01-01 04:51:00
            HourlyWindSpeed  HourlyPrecipitation
DATE                                            
2009-01-01        11.041667                  0.0
2009-01-02         6.083333                  0.0
2009-01-03         9.875000                  0.0
2009-01-

In [29]:
hourly_weather_data.head()

,DATE,HourlyWindSpeed,HourlyPrecipitation,ndate
0,2009-01-01,18.0,0.0,2009-01-01 00:51:00
1,2009-01-01,18.0,0.0,2009-01-01 01:51:00
2,2009-01-01,18.0,0.0,2009-01-01 02:51:00
3,2009-01-01,8.0,0.0,2009-01-01 03:51:00
4,2009-01-01,11.0,0.0,2009-01-01 04:51:00


In [30]:
daily_weather_data.head()

,HourlyWindSpeed,HourlyPrecipitation
DATE,,
2009-01-01,11.041667,0.0
2009-01-02,6.083333,0.0
2009-01-03,9.875000,0.0
2009-01-04,7.416667,0.0
2009-01-05,7.000000,0.0


## Part 2: Storing Cleaned Data

In [31]:
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///projects.db', echo=False)


### Add Data to Database

In [32]:
def write_yellow(df):
    #df=get_add_yellowtaxi()
    df.to_sql('taxitips',con=engine,if_exists="append")


In [33]:
def write_userb(df=None):
    if df==None:
        df=get_uberdata(uber_path)
    df.to_sql('ubertips',con=engine,if_exists="append")


In [34]:
def write_hourlyweather(df1):
    # df1,df2=load_and_clean_weather_data(weather_path1)

    df1.to_sql('hourlyweather',con=engine,if_exists="append")

def write_daylyweather(df2):
    # df1,df2=load_and_clean_weather_data(weather_path1)
    df2.to_sql('daylyweather',con=engine,if_exists="append")


In [35]:
write_yellow(get_add_yellowtaxi())
write_userb()
write_weather()

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-03.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-04.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-05.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-06.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-07.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-08.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-09.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-10.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-11.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-12.parquet', 'ht

FileNotFoundError: [Errno 2] No such file or directory: '/Users/guowenlu/Desktop/TFA/data/queriesyellow_tripdata_2015-01.parquet'